In [ ]:
import os, sys

import pymongo as pm
import numpy as np
import scipy.stats as stats
import pandas as pd
import json
import re
from io import BytesIO
from PIL import Image
import base64
import PIL

import matplotlib
from matplotlib import pylab, mlab, pyplot
import matplotlib.patches as mpatches
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('talk')
sns.set_style('white')

import math

from IPython.display import clear_output
import importlib

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")


### setup paths

In [ ]:
# directory & file hierarchy
proj_dir = os.path.abspath('..')
analysis_dir = os.getcwd()
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
sketch_dir = os.path.abspath(os.path.join(proj_dir,'sketches'))
gallery_dir = os.path.abspath(os.path.join(proj_dir,'gallery'))

## add helpers to python path
if os.path.join(proj_dir,'utils') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'utils'))   

def make_dir_if_not_exists(dir_name):   
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    return dir_name

## create directories that don't already exist        
result = [make_dir_if_not_exists(x) for x in [results_dir,plot_dir,csv_dir,sketch_dir,gallery_dir]]

## add utils to python path
import sys
if os.path.join(proj_dir,'utils') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'utils'))
import utils

### set up connection to mongo

In [ ]:
# set vars 
auth = pd.read_csv(os.path.join(analysis_dir,'auth.txt'), header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org'

# have to fix this to be able to analyze from local
import pymongo as pm
import socket
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1:27017')
db = conn['iterated_number']
coll = db['num8_shape4']

# which iteration name should we use?
iterationName = 'run6' #increment when needed


In [ ]:
# get all the gameids
all_games = coll.distinct('gameid')

In [ ]:
# sort all the recs in descending order (so most recent to least recent)
recs = coll.find({'iterationName':iterationName}).sort('time', -1)
# grab the first one (i.e., the most recent) and print out the gameid
most_recent_gameid = recs[0]['gameid'] 
print('The most recent gameid is {}.'.format(most_recent_gameid))

In [ ]:

def bonus_finder(iteration_name,workerID):
    """
    1. Quickly looks in mongo for the worker ID.
    2. Gets the last trial they participated in.
    3. Applies the following function to compute their bonus (may vary per experiment):
    bonus = (previous_score + previous_bonus_score) / 100
    
    """
    need_money = coll.find({'iterationName':iteration_name, 'workerId':workerID})
    Need_money = pd.DataFrame(need_money)
    bonus_row = Need_money[['gameid','trialNum','previous_score','previous_bonus_score']].tail(1)
    bonus = math.ceil(float(bonus_row['previous_score'].values[0]) + float(bonus_row['previous_bonus_score'].values[0])) / 100     
    print(bonus_row)
    return bonus

# bonus_finder(iterationName,'workerID')
